# Computational Theory

In [9]:
import numpy as np

## Problem 1: Binary Words and Operations

### Parity Function

""" 

"""

In [ ]:
def Parity(x, y, z):
    """Return the bitwise parity (XOR) of three 32-bit integers."""
    # The numpy.int32 type ensures that the inputs are treated as 32-bit integers.
    # This is important because Python's default int type can be of arbitrary precision.
    # SHA-256 , SHA-1 and SHA-224 all specifically require 32-bit words.
    # See : https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.int32
    x = np.int32(x)
    y = np.int32(y)
    z = np.int32(z)
    return np.uint32(x ^ y ^ z)


In [ ]:
def Ch(x,y,z):
    """Returns the bitwise choice function result for three 32-bit integers.

    """
    x = np.int32(x)
    y = np.int32(y)
    z = np.int32(z)
    return np.uint32((x & y) ^ (~x & z))


In [3]:
def Maj(x,y,z):
    """Return the bitwise majority function result for three 32-bit integers."""
    x = np.int32(x)
    y = np.int32(y)
    z = np.int32(z)
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

In [4]:
def Sigma0(x):
    """Return the Sigma0 function used in SHA-256 for a 32-bit integer."""
    x = np.int32(x)
    return np.uint32((x >> 2) ^ (x >> 13) ^ (x >> 22))



In [5]:
def Sigma1(x):
    """Return the Sigma1 function used in SHA-256 for a 32-bit integer."""
    x = np.int32(x)
    return np.uint32((x >> 6) ^ (x >> 11) ^ (x >> 25))


In [6]:
def sigma0(x):
    """Return the sigma0 function used in SHA-256 for a 32-bit integer."""
    x = np.int32(x)
    return np.uint32((x >> 7) ^ (x >> 18) ^ (x >> 3))


In [7]:
def sigma1(x):
    """Return the sigma1 function used in SHA-256 for a 32-bit integer."""
    x = np.int32(x)
    return np.uint32((x >> 17) ^ (x >> 19) ^ (x >> 10))


In [ ]:
"""Example usage of the functions."""

print(Parity(0b1100, 0b1010, 0b1001))  # Example usage
print(Ch(0b1100, 0b1010, 0b1001))      # Example usage
print(Maj(0b1100, 0b1010, 0b1001))     # Example usage
print(Sigma0(0b1100))               # Example usage
print(Sigma1(0b1100))               # Example usage
print(sigma0(0b1100))               # Example usage
print(sigma1(0b1100))               # Example usage

15
9
8
3
0
1
0


## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords

## End